In [1]:
import sqlite3, pandas as pd, time
from IPython.display import display

conn = sqlite3.connect("mini_shop.db")
conn.execute("PRAGMA foreign_keys = ON;")
conn.execute("PRAGMA recursive_triggers = OFF;")  
run = conn.executescript

In [2]:
run("""
-- cleanup
DROP VIEW  IF EXISTS v_product_current_price;

DROP TRIGGER IF EXISTS tr_products_price_insert;
DROP TRIGGER IF EXISTS tr_products_price_update;
DROP TRIGGER IF EXISTS tr_products_touch;
DROP TRIGGER IF EXISTS tr_product_audit_upd;
DROP TRIGGER IF EXISTS tr_product_soft_delete;

DROP TRIGGER IF EXISTS tr_users_touch;
DROP TRIGGER IF EXISTS tr_users_prev_country;

DROP TRIGGER IF EXISTS tr_oi_validate_not_deleted_ins;
DROP TRIGGER IF EXISTS tr_oi_validate_not_deleted_upd;
DROP TRIGGER IF EXISTS tr_oi_after_insert;
DROP TRIGGER IF EXISTS tr_oi_after_update;
DROP TRIGGER IF EXISTS tr_oi_after_delete;

DROP TRIGGER IF EXISTS tr_orders_touch;
DROP TRIGGER IF EXISTS tr_orders_recalc_after_coupon;
DROP TRIGGER IF EXISTS tr_orders_validate_coupon_on_apply;
DROP TRIGGER IF EXISTS tr_orders_validate_coupon_on_paid;
DROP TRIGGER IF EXISTS tr_orders_log_redemption_paid;
DROP TRIGGER IF EXISTS tr_orders_log_redemption_cancel;

DROP TABLE IF EXISTS coupon_redemptions;
DROP TABLE IF EXISTS coupons;
DROP TABLE IF EXISTS order_items;
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS product_price_history;
DROP TABLE IF EXISTS product_audit;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS users;

-- users (SCD3 prev_country)
CREATE TABLE users(
  user_id      INTEGER PRIMARY KEY AUTOINCREMENT,
  email        TEXT UNIQUE NOT NULL,
  country      TEXT NOT NULL,
  prev_country TEXT,
  created_at   TEXT NOT NULL DEFAULT CURRENT_TIMESTAMP,
  updated_at   TEXT NOT NULL DEFAULT CURRENT_TIMESTAMP,
  row_version  INTEGER NOT NULL DEFAULT 1
);

-- products (SCD1 name overwrite, SCD2 price history, SCD3 prev_price, soft-delete)
CREATE TABLE products(
  product_id   INTEGER PRIMARY KEY AUTOINCREMENT,
  sku          TEXT UNIQUE NOT NULL,
  name         TEXT NOT NULL,
  price        REAL NOT NULL CHECK(price >= 0),
  prev_price   REAL,
  is_deleted   INTEGER NOT NULL DEFAULT 0 CHECK(is_deleted IN (0,1)),
  deleted_at   TEXT,
  created_at   TEXT NOT NULL DEFAULT CURRENT_TIMESTAMP,
  updated_at   TEXT NOT NULL DEFAULT CURRENT_TIMESTAMP,
  row_version  INTEGER NOT NULL DEFAULT 1
);

-- SCD2 price history
CREATE TABLE product_price_history(
  id          INTEGER PRIMARY KEY AUTOINCREMENT,
  product_id  INTEGER NOT NULL REFERENCES products(product_id) ON DELETE CASCADE,
  price       REAL NOT NULL,
  valid_from  TEXT NOT NULL,
  valid_to    TEXT,
  is_current  INTEGER NOT NULL DEFAULT 1 CHECK(is_current IN (0,1))
);

-- audit
CREATE TABLE product_audit(
  log_id      INTEGER PRIMARY KEY AUTOINCREMENT,
  product_id  INTEGER NOT NULL,
  action      TEXT NOT NULL CHECK(action IN ('INSERT','UPDATE','SOFT_DELETE')),
  changed_at  TEXT NOT NULL DEFAULT CURRENT_TIMESTAMP,
  old_name    TEXT, new_name TEXT,
  old_price   REAL, new_price REAL
);

-- coupons
CREATE TABLE coupons(
  coupon_id           INTEGER PRIMARY KEY AUTOINCREMENT,
  code                TEXT UNIQUE NOT NULL,
  type                TEXT NOT NULL CHECK(type IN ('percent','fixed')),
  value               REAL NOT NULL CHECK(value > 0 AND (type <> 'percent' OR value <= 100)),
  is_active           INTEGER NOT NULL DEFAULT 1 CHECK(is_active IN (0,1)),
  valid_from          TEXT,   -- NULL = нет ограничения
  valid_to            TEXT,   -- NULL = нет ограничения
  min_order_amount    REAL NOT NULL DEFAULT 0 CHECK(min_order_amount >= 0),
  max_uses            INTEGER, -- NULL = без лимита
  max_uses_per_user   INTEGER  -- NULL = без лимита
);

CREATE TABLE coupon_redemptions(
  redemption_id INTEGER PRIMARY KEY AUTOINCREMENT,
  coupon_id     INTEGER NOT NULL REFERENCES coupons(coupon_id) ON DELETE CASCADE,
  order_id      INTEGER NOT NULL REFERENCES orders(order_id)   ON DELETE CASCADE,
  user_id       INTEGER NOT NULL REFERENCES users(user_id)     ON DELETE CASCADE,
  redeemed_at   TEXT NOT NULL DEFAULT CURRENT_TIMESTAMP,
  UNIQUE(coupon_id, order_id)
);

-- orders (+ купон и суммы)
CREATE TABLE orders(
  order_id       INTEGER PRIMARY KEY AUTOINCREMENT,
  user_id        INTEGER NOT NULL REFERENCES users(user_id),
  status         TEXT NOT NULL DEFAULT 'new' CHECK(status IN ('new','paid','shipped','cancelled')),
  coupon_id      INTEGER REFERENCES coupons(coupon_id),
  subtotal       REAL NOT NULL DEFAULT 0,
  discount_amount REAL NOT NULL DEFAULT 0,
  total_amount   REAL NOT NULL DEFAULT 0,
  created_at     TEXT NOT NULL DEFAULT CURRENT_TIMESTAMP,
  updated_at     TEXT NOT NULL DEFAULT CURRENT_TIMESTAMP,
  row_version    INTEGER NOT NULL DEFAULT 1
);

-- order_items
CREATE TABLE order_items(
  order_item_id     INTEGER PRIMARY KEY AUTOINCREMENT,
  order_id          INTEGER NOT NULL REFERENCES orders(order_id) ON DELETE CASCADE,
  product_id        INTEGER NOT NULL REFERENCES products(product_id),
  qty               INTEGER NOT NULL CHECK(qty > 0),
  price_at_purchase REAL NOT NULL DEFAULT 0,
  amount            REAL NOT NULL DEFAULT 0
);

CREATE INDEX idx_orders_user ON orders(user_id);
CREATE INDEX idx_orders_coupon ON orders(coupon_id);
CREATE INDEX idx_oi_order ON order_items(order_id);
CREATE INDEX idx_oi_product ON order_items(product_id);
CREATE INDEX idx_hist_curr ON product_price_history(product_id, is_current);
CREATE INDEX idx_redempt_coupon ON coupon_redemptions(coupon_id);
CREATE INDEX idx_redempt_user ON coupon_redemptions(user_id);

-- view текущих цен
CREATE VIEW v_product_current_price AS
SELECT p.product_id, p.sku, p.name, h.price AS current_price, p.is_deleted, p.updated_at
FROM products p
JOIN product_price_history h
  ON h.product_id = p.product_id AND h.is_current = 1;

-- ===== триггеры =====

-- products meta
CREATE TRIGGER tr_products_touch
AFTER UPDATE ON products
BEGIN
  UPDATE products
    SET updated_at = CURRENT_TIMESTAMP,
        row_version = row_version + 1
  WHERE product_id = NEW.product_id;
END;

-- SCD2: при INSERT продукта
CREATE TRIGGER tr_products_price_insert
AFTER INSERT ON products
BEGIN
  INSERT INTO product_price_history(product_id, price, valid_from, valid_to, is_current)
  VALUES (NEW.product_id, NEW.price, CURRENT_TIMESTAMP, NULL, 1);

  INSERT INTO product_audit(product_id, action, old_name, new_name, old_price, new_price)
  VALUES (NEW.product_id, 'INSERT', NULL, NEW.name, NULL, NEW.price);
END;

-- SCD2+SCD3: при смене цены
CREATE TRIGGER tr_products_price_update
AFTER UPDATE OF price ON products
BEGIN
  UPDATE product_price_history
     SET valid_to = CURRENT_TIMESTAMP, is_current = 0
   WHERE product_id = NEW.product_id AND is_current = 1;

  INSERT INTO product_price_history(product_id, price, valid_from, valid_to, is_current)
  VALUES (NEW.product_id, NEW.price, CURRENT_TIMESTAMP, NULL, 1);

  UPDATE products SET prev_price = OLD.price WHERE product_id = NEW.product_id;

  INSERT INTO product_audit(product_id, action, old_name, new_name, old_price, new_price)
  VALUES (NEW.product_id, 'UPDATE', OLD.name, NEW.name, OLD.price, NEW.price);
END;

-- SCD1: переименование (аудит)
CREATE TRIGGER tr_product_audit_upd
AFTER UPDATE OF name ON products
BEGIN
  INSERT INTO product_audit(product_id, action, old_name, new_name, old_price, new_price)
  VALUES (NEW.product_id, 'UPDATE', OLD.name, NEW.name, OLD.price, NEW.price);
END;

-- SOFT DELETE: вместо DELETE ставим флаг и логируем
CREATE TRIGGER tr_product_soft_delete
BEFORE DELETE ON products
BEGIN
  UPDATE products
     SET is_deleted = 1,
         deleted_at = CURRENT_TIMESTAMP
   WHERE product_id = OLD.product_id;

  INSERT INTO product_audit(product_id, action, old_name, new_name, old_price, new_price)
  VALUES (OLD.product_id, 'SOFT_DELETE', OLD.name, NULL, OLD.price, NULL);

  SELECT RAISE(IGNORE);
END;

-- users meta + SCD3 (prev_country)
CREATE TRIGGER tr_users_touch
AFTER UPDATE ON users
BEGIN
  UPDATE users
    SET updated_at = CURRENT_TIMESTAMP,
        row_version = row_version + 1
  WHERE user_id = NEW.user_id;
END;

CREATE TRIGGER tr_users_prev_country
AFTER UPDATE OF country ON users
BEGIN
  UPDATE users SET prev_country = OLD.country WHERE user_id = NEW.user_id;
END;

-- Нельзя добавлять в заказ удалённые продукты
CREATE TRIGGER tr_oi_validate_not_deleted_ins
BEFORE INSERT ON order_items
BEGIN
  SELECT CASE
    WHEN (SELECT is_deleted FROM products WHERE product_id = NEW.product_id) = 1
    THEN RAISE(ABORT, 'product is soft-deleted')
  END;
END;

CREATE TRIGGER tr_oi_validate_not_deleted_upd
BEFORE UPDATE OF product_id ON order_items
BEGIN
  SELECT CASE
    WHEN (SELECT is_deleted FROM products WHERE product_id = NEW.product_id) = 1
    THEN RAISE(ABORT, 'product is soft-deleted')
  END;
END;

-- Пересчёт суммы позиции и заказа
CREATE TRIGGER tr_oi_after_insert
AFTER INSERT ON order_items
BEGIN
  UPDATE order_items
     SET price_at_purchase = (SELECT price FROM products WHERE product_id = NEW.product_id),
         amount            = (SELECT price FROM products WHERE product_id = NEW.product_id) * NEW.qty
   WHERE order_item_id = NEW.order_item_id;

  -- пересчёт заказа с учётом купона
  UPDATE orders
     SET subtotal = (SELECT COALESCE(SUM(amount),0) FROM order_items WHERE order_id = NEW.order_id),
         discount_amount = (
           WITH sub AS (SELECT COALESCE(SUM(amount),0) AS s FROM order_items WHERE order_id = NEW.order_id)
           SELECT CASE
             WHEN orders.coupon_id IS NULL THEN 0
             WHEN NOT EXISTS (
               SELECT 1 FROM coupons c
                WHERE c.coupon_id = orders.coupon_id
                  AND c.is_active = 1
                  AND (c.valid_from IS NULL OR c.valid_from <= CURRENT_TIMESTAMP)
                  AND (c.valid_to   IS NULL OR c.valid_to   >  CURRENT_TIMESTAMP)
                  AND s >= COALESCE(c.min_order_amount,0)
             ) THEN 0
             ELSE (SELECT CASE c.type
                          WHEN 'percent' THEN ROUND(s * c.value/100.0, 2)
                          WHEN 'fixed'   THEN MIN(c.value, s)
                        END
                   FROM coupons c WHERE c.coupon_id = orders.coupon_id)
           END FROM sub
         ),
         total_amount = (
           WITH sub AS (SELECT COALESCE(SUM(amount),0) AS s FROM order_items WHERE order_id = NEW.order_id),
                disc AS (
                  SELECT CASE
                    WHEN orders.coupon_id IS NULL THEN 0
                    WHEN NOT EXISTS (
                      SELECT 1 FROM coupons c
                       WHERE c.coupon_id = orders.coupon_id
                         AND c.is_active = 1
                         AND (c.valid_from IS NULL OR c.valid_from <= CURRENT_TIMESTAMP)
                         AND (c.valid_to   IS NULL OR c.valid_to   >  CURRENT_TIMESTAMP)
                         AND s >= COALESCE(c.min_order_amount,0)
                    ) THEN 0
                    ELSE (SELECT CASE c.type
                                 WHEN 'percent' THEN ROUND(s * c.value/100.0, 2)
                                 WHEN 'fixed'   THEN MIN(c.value, s)
                               END
                          FROM coupons c WHERE c.coupon_id = orders.coupon_id)
                  END AS d FROM sub
                )
           SELECT MAX(s - d, 0) FROM sub, disc
         ),
         updated_at = CURRENT_TIMESTAMP,
         row_version = row_version + 1
   WHERE order_id = NEW.order_id;
END;

CREATE TRIGGER tr_oi_after_update
AFTER UPDATE OF qty, product_id ON order_items
BEGIN
  UPDATE order_items
     SET price_at_purchase = (SELECT price FROM products WHERE product_id = NEW.product_id),
         amount            = (SELECT price FROM products WHERE product_id = NEW.product_id) * NEW.qty
   WHERE order_item_id = NEW.order_item_id;

  UPDATE orders
     SET subtotal = (SELECT COALESCE(SUM(amount),0) FROM order_items WHERE order_id = NEW.order_id),
         discount_amount = (
           WITH sub AS (SELECT COALESCE(SUM(amount),0) AS s FROM order_items WHERE order_id = NEW.order_id)
           SELECT CASE
             WHEN orders.coupon_id IS NULL THEN 0
             WHEN NOT EXISTS (
               SELECT 1 FROM coupons c
                WHERE c.coupon_id = orders.coupon_id
                  AND c.is_active = 1
                  AND (c.valid_from IS NULL OR c.valid_from <= CURRENT_TIMESTAMP)
                  AND (c.valid_to   IS NULL OR c.valid_to   >  CURRENT_TIMESTAMP)
                  AND s >= COALESCE(c.min_order_amount,0)
             ) THEN 0
             ELSE (SELECT CASE c.type
                          WHEN 'percent' THEN ROUND(s * c.value/100.0, 2)
                          WHEN 'fixed'   THEN MIN(c.value, s)
                        END
                   FROM coupons c WHERE c.coupon_id = orders.coupon_id)
           END FROM sub
         ),
         total_amount = (
           WITH sub AS (SELECT COALESCE(SUM(amount),0) AS s FROM order_items WHERE order_id = NEW.order_id),
                disc AS (
                  SELECT CASE
                    WHEN orders.coupon_id IS NULL THEN 0
                    WHEN NOT EXISTS (
                      SELECT 1 FROM coupons c
                       WHERE c.coupon_id = orders.coupon_id
                         AND c.is_active = 1
                         AND (c.valid_from IS NULL OR c.valid_from <= CURRENT_TIMESTAMP)
                         AND (c.valid_to   IS NULL OR c.valid_to   >  CURRENT_TIMESTAMP)
                         AND s >= COALESCE(c.min_order_amount,0)
                    ) THEN 0
                    ELSE (SELECT CASE c.type
                                 WHEN 'percent' THEN ROUND(s * c.value/100.0, 2)
                                 WHEN 'fixed'   THEN MIN(c.value, s)
                               END
                          FROM coupons c WHERE c.coupon_id = orders.coupon_id)
                  END AS d FROM sub
                )
           SELECT MAX(s - d, 0) FROM sub, disc
         ),
         updated_at = CURRENT_TIMESTAMP,
         row_version = row_version + 1
   WHERE order_id = NEW.order_id;
END;

CREATE TRIGGER tr_oi_after_delete
AFTER DELETE ON order_items
BEGIN
  UPDATE orders
     SET subtotal = (SELECT COALESCE(SUM(amount),0) FROM order_items WHERE order_id = OLD.order_id),
         discount_amount = (
           WITH sub AS (SELECT COALESCE(SUM(amount),0) AS s FROM order_items WHERE order_id = OLD.order_id)
           SELECT CASE
             WHEN orders.coupon_id IS NULL THEN 0
             WHEN NOT EXISTS (
               SELECT 1 FROM coupons c
                WHERE c.coupon_id = orders.coupon_id
                  AND c.is_active = 1
                  AND (c.valid_from IS NULL OR c.valid_from <= CURRENT_TIMESTAMP)
                  AND (c.valid_to   IS NULL OR c.valid_to   >  CURRENT_TIMESTAMP)
                  AND s >= COALESCE(c.min_order_amount,0)
             ) THEN 0
             ELSE (SELECT CASE c.type
                          WHEN 'percent' THEN ROUND(s * c.value/100.0, 2)
                          WHEN 'fixed'   THEN MIN(c.value, s)
                        END
                   FROM coupons c WHERE c.coupon_id = orders.coupon_id)
           END FROM sub
         ),
         total_amount = (
           WITH sub AS (SELECT COALESCE(SUM(amount),0) AS s FROM order_items WHERE order_id = OLD.order_id),
                disc AS (
                  SELECT CASE
                    WHEN orders.coupon_id IS NULL THEN 0
                    WHEN NOT EXISTS (
                      SELECT 1 FROM coupons c
                       WHERE c.coupon_id = orders.coupon_id
                         AND c.is_active = 1
                         AND (c.valid_from IS NULL OR c.valid_from <= CURRENT_TIMESTAMP)
                         AND (c.valid_to   IS NULL OR c.valid_to   >  CURRENT_TIMESTAMP)
                         AND s >= COALESCE(c.min_order_amount,0)
                    ) THEN 0
                    ELSE (SELECT CASE c.type
                                 WHEN 'percent' THEN ROUND(s * c.value/100.0, 2)
                                 WHEN 'fixed'   THEN MIN(c.value, s)
                               END
                          FROM coupons c WHERE c.coupon_id = orders.coupon_id)
                  END AS d FROM sub
                )
           SELECT MAX(s - d, 0) FROM sub, disc
         ),
         updated_at = CURRENT_TIMESTAMP,
         row_version = row_version + 1
   WHERE order_id = OLD.order_id;
END;

-- общий touch (на случай ручных апдейтов)
CREATE TRIGGER tr_orders_touch
AFTER UPDATE ON orders
BEGIN
  UPDATE orders
    SET updated_at = CURRENT_TIMESTAMP,
        row_version = row_version + 1
  WHERE order_id = NEW.order_id;
END;

-- смена купона -> пересчёт
CREATE TRIGGER tr_orders_recalc_after_coupon
AFTER UPDATE OF coupon_id ON orders
BEGIN
  UPDATE orders
     SET subtotal = (SELECT COALESCE(SUM(amount),0) FROM order_items WHERE order_id = NEW.order_id),
         discount_amount = (
           WITH sub AS (SELECT COALESCE(SUM(amount),0) AS s FROM order_items WHERE order_id = NEW.order_id)
           SELECT CASE
             WHEN orders.coupon_id IS NULL THEN 0
             WHEN NOT EXISTS (
               SELECT 1 FROM coupons c
                WHERE c.coupon_id = orders.coupon_id
                  AND c.is_active = 1
                  AND (c.valid_from IS NULL OR c.valid_from <= CURRENT_TIMESTAMP)
                  AND (c.valid_to   IS NULL OR c.valid_to   >  CURRENT_TIMESTAMP)
                  AND s >= COALESCE(c.min_order_amount,0)
             ) THEN 0
             ELSE (SELECT CASE c.type
                          WHEN 'percent' THEN ROUND(s * c.value/100.0, 2)
                          WHEN 'fixed'   THEN MIN(c.value, s)
                        END
                   FROM coupons c WHERE c.coupon_id = orders.coupon_id)
           END FROM sub
         ),
         total_amount = (
           WITH sub AS (SELECT COALESCE(SUM(amount),0) AS s FROM order_items WHERE order_id = NEW.order_id),
                disc AS (
                  SELECT CASE
                    WHEN orders.coupon_id IS NULL THEN 0
                    WHEN NOT EXISTS (
                      SELECT 1 FROM coupons c
                       WHERE c.coupon_id = orders.coupon_id
                         AND c.is_active = 1
                         AND (c.valid_from IS NULL OR c.valid_from <= CURRENT_TIMESTAMP)
                         AND (c.valid_to   IS NULL OR c.valid_to   >  CURRENT_TIMESTAMP)
                         AND s >= COALESCE(c.min_order_amount,0)
                    ) THEN 0
                    ELSE (SELECT CASE c.type
                                 WHEN 'percent' THEN ROUND(s * c.value/100.0, 2)
                                 WHEN 'fixed'   THEN MIN(c.value, s)
                               END
                          FROM coupons c WHERE c.coupon_id = orders.coupon_id)
                  END AS d FROM sub
                )
           SELECT MAX(s - d, 0) FROM sub, disc
         ),
         updated_at = CURRENT_TIMESTAMP,
         row_version = row_version + 1
   WHERE order_id = NEW.order_id;
END;

-- валидация купона при применении к заказу
CREATE TRIGGER tr_orders_validate_coupon_on_apply
BEFORE UPDATE OF coupon_id ON orders
BEGIN
  -- активность и валидность по датам
  SELECT CASE
    WHEN NEW.coupon_id IS NOT NULL AND NOT EXISTS (
      SELECT 1 FROM coupons c
       WHERE c.coupon_id = NEW.coupon_id
         AND c.is_active = 1
         AND (c.valid_from IS NULL OR c.valid_from <= CURRENT_TIMESTAMP)
         AND (c.valid_to   IS NULL OR c.valid_to   >  CURRENT_TIMESTAMP)
    )
    THEN RAISE(ABORT, 'coupon inactive or expired')
  END;

  -- минимальная сумма заказа
  SELECT CASE
    WHEN NEW.coupon_id IS NOT NULL AND
         (SELECT COALESCE(SUM(amount),0) FROM order_items WHERE order_id = NEW.order_id)
         < (SELECT COALESCE(min_order_amount,0) FROM coupons WHERE coupon_id = NEW.coupon_id)
    THEN RAISE(ABORT, 'order sum below coupon minimum')
  END;
END;

-- проверка лимитов при оплате
CREATE TRIGGER tr_orders_validate_coupon_on_paid
BEFORE UPDATE OF status ON orders
WHEN NEW.status = 'paid' AND NEW.coupon_id IS NOT NULL
BEGIN
  -- общий лимит
  SELECT CASE
    WHEN (SELECT COUNT(*) FROM coupon_redemptions WHERE coupon_id = NEW.coupon_id)
         >= COALESCE((SELECT max_uses FROM coupons WHERE coupon_id = NEW.coupon_id), 999999999)
    THEN RAISE(ABORT, 'coupon usage limit reached')
  END;

  -- лимит на пользователя
  SELECT CASE
    WHEN (SELECT COUNT(*) FROM coupon_redemptions WHERE coupon_id = NEW.coupon_id AND user_id = NEW.user_id)
         >= COALESCE((SELECT max_uses_per_user FROM coupons WHERE coupon_id = NEW.coupon_id), 999999999)
    THEN RAISE(ABORT, 'coupon per-user limit reached')
  END;
END;

-- логирование редемпшна
CREATE TRIGGER tr_orders_log_redemption_paid
AFTER UPDATE OF status ON orders
WHEN NEW.status = 'paid' AND NEW.coupon_id IS NOT NULL
BEGIN
  INSERT OR IGNORE INTO coupon_redemptions(coupon_id, order_id, user_id)
  VALUES (NEW.coupon_id, NEW.order_id, NEW.user_id);
END;

-- отмена: снимаем редемпшн
CREATE TRIGGER tr_orders_log_redemption_cancel
AFTER UPDATE OF status ON orders
WHEN NEW.status = 'cancelled'
BEGIN
  DELETE FROM coupon_redemptions WHERE order_id = NEW.order_id;
END;
""")
print("Schema + triggers ready")

Schema + triggers ready


In [3]:
# продукты
conn.executemany("INSERT INTO products(sku, name, price) VALUES (?,?,?)",
                 [("SKU-TSHIRT","T-Shirt",20.0),
                  ("SKU-MUG","Mug",10.0),
                  ("SKU-HOOD","Hoodie",45.0),
                  ("SKU-CAP","Cap",15.0)])
# пользователи
conn.executemany("INSERT INTO users(email, country) VALUES (?,?)",
                 [("alice@example.com","RU"),
                  ("bob@example.com","US"),
                  ("carol@example.com","DE")])
conn.commit()

# SCD2: смена цены у T-Shirt
ts_before = pd.read_sql("SELECT CURRENT_TIMESTAMP ts", conn).ts[0]
time.sleep(0.8)
conn.execute("UPDATE products SET price=22.5 WHERE sku='SKU-TSHIRT';")
# SCD1: переименование кружки
conn.execute("UPDATE products SET name='Coffee Mug' WHERE sku='SKU-MUG';")
# SCD3: страна у Alice
conn.execute("UPDATE users SET country='US' WHERE email='alice@example.com';")
conn.commit()
ts_after = pd.read_sql("SELECT CURRENT_TIMESTAMP ts", conn).ts[0]

# купоны
run("""
INSERT INTO coupons(code,type,value,is_active,valid_from,valid_to,min_order_amount,max_uses,max_uses_per_user)
VALUES
 ('WELCOME10','percent',10,1,DATETIME('now','-1 day'),NULL,20,NULL,1),
 ('FIVEOFF','fixed',5,1,DATETIME('now','-1 day'),NULL,10,NULL,NULL),
 ('SUMMER30','percent',30,1,DATETIME('now','-1 day'),DATETIME('now','+30 day'),30,2,NULL);
""")
print("Seed done")

Seed done


In [4]:
# Order 1 (Alice): 2 футболки + кружка, потом применим WELCOME10
conn.execute("INSERT INTO orders(user_id, status) VALUES ((SELECT user_id FROM users WHERE email='alice@example.com'),'new');")
order1 = conn.execute("SELECT last_insert_rowid()").fetchone()[0]
conn.executemany(
    "INSERT INTO order_items(order_id, product_id, qty) VALUES (?,?,?)",
    [
        (order1, conn.execute("SELECT product_id FROM products WHERE sku='SKU-TSHIRT'").fetchone()[0], 2),
        (order1, conn.execute("SELECT product_id FROM products WHERE sku='SKU-MUG'").fetchone()[0], 1),
    ]
)
conn.execute("""
UPDATE orders
SET coupon_id = (SELECT coupon_id FROM coupons WHERE code='WELCOME10')
WHERE order_id = ?;
""", (order1,))
# Оплата -> редемпшн
conn.execute("UPDATE orders SET status='paid' WHERE order_id=?", (order1,))

# Order 2 (Bob): худи, купон FIVEOFF
conn.execute("INSERT INTO orders(user_id, status) VALUES ((SELECT user_id FROM users WHERE email='bob@example.com'),'new');")
order2 = conn.execute("SELECT last_insert_rowid()").fetchone()[0]
conn.execute("INSERT INTO order_items(order_id, product_id, qty) VALUES (?,?,?)",
             (order2, conn.execute("SELECT product_id FROM products WHERE sku='SKU-HOOD'").fetchone()[0], 1))
conn.execute("""
UPDATE orders SET coupon_id=(SELECT coupon_id FROM coupons WHERE code='FIVEOFF') WHERE order_id=?;
""", (order2,))
conn.execute("UPDATE orders SET status='paid' WHERE order_id=?", (order2,))
conn.commit()

# Soft-delete продукта 'MUG'
conn.execute("DELETE FROM products WHERE sku='SKU-MUG';")  # фактического удаления не будет
conn.commit()

# попробуем добавить удалённый продукт в заказ (должно упасть)
try:
    mug_id = conn.execute("SELECT product_id FROM products WHERE sku='SKU-MUG'").fetchone()
    print("MUG row exists after soft-delete? ->", mug_id is not None)
    conn.execute("INSERT INTO order_items(order_id, product_id, qty) VALUES (?,?,?)",
                 (order2, conn.execute("SELECT product_id FROM products WHERE sku='SKU-MUG'").fetchone()[0], 1))
except Exception as e:
    print("Ожидаемая ошибка при добавлении soft-deleted товара:", e)

MUG row exists after soft-delete? -> True
Ожидаемая ошибка при добавлении soft-deleted товара: product is soft-deleted


In [5]:
print("Текущие цены:")
display(pd.read_sql("SELECT * FROM v_product_current_price ORDER BY sku", conn))

print("История цен T-Shirt (SCD2):")
display(pd.read_sql("""
  SELECT price, valid_from, valid_to, is_current
  FROM product_price_history
  WHERE product_id=(SELECT product_id FROM products WHERE sku='SKU-TSHIRT')
  ORDER BY valid_from
""", conn))

print("Цена T-Shirt на ts_before и ts_after:")
display(pd.read_sql("""
SELECT
  (SELECT price FROM product_price_history WHERE product_id=(SELECT product_id FROM products WHERE sku='SKU-TSHIRT')
   AND valid_from<=? AND (valid_to IS NULL OR valid_to>?) ORDER BY valid_from DESC LIMIT 1) AS price_before,
  (SELECT price FROM product_price_history WHERE product_id=(SELECT product_id FROM products WHERE sku='SKU-TSHIRT')
   AND valid_from<=? AND (valid_to IS NULL OR valid_to>?) ORDER BY valid_from DESC LIMIT 1) AS price_after
""", conn, params=[ts_before, ts_before, ts_after, ts_after]))

print("Статус soft-delete по продуктам:")
display(pd.read_sql("SELECT sku, name, is_deleted, deleted_at FROM products ORDER BY sku", conn))

print("Купоны:")
display(pd.read_sql("SELECT code, type, value, is_active, valid_from, valid_to, min_order_amount, max_uses, max_uses_per_user FROM coupons", conn))

print("Заказы (subtotal/discount/total):")
display(pd.read_sql("""
  SELECT o.order_id, u.email, o.status, o.subtotal, o.discount_amount, o.total_amount,
         (SELECT code FROM coupons c WHERE c.coupon_id=o.coupon_id) AS coupon
  FROM orders o JOIN users u ON o.user_id=u.user_id
  ORDER BY o.order_id
""", conn))

print("Редемпшены купонов:")
display(pd.read_sql("""
  SELECT r.order_id, u.email, c.code, r.redeemed_at
  FROM coupon_redemptions r
  JOIN users u ON r.user_id=u.user_id
  JOIN coupons c ON c.coupon_id=r.coupon_id
  ORDER BY r.redemption_id
""", conn))

Текущие цены:


,product_id,sku,name,current_price,is_deleted,updated_at
0,4,SKU-CAP,Cap,15.0,0,2025-08-26 17:10:00
1,3,SKU-HOOD,Hoodie,45.0,0,2025-08-26 17:10:00
2,2,SKU-MUG,Coffee Mug,10.0,1,2025-08-26 17:10:03
3,1,SKU-TSHIRT,T-Shirt,22.5,0,2025-08-26 17:10:01


История цен T-Shirt (SCD2):


,price,valid_from,valid_to,is_current
0,20.0,2025-08-26 17:10:00,2025-08-26 17:10:01,0
1,22.5,2025-08-26 17:10:01,None,1


Цена T-Shirt на ts_before и ts_after:


...

,code,type,value,is_active,valid_from,valid_to,min_order_amount,max_uses,max_uses_per_user
0,WELCOME10,percent,10.0,1,2025-08-25 17:10:01,None,20.0,NaN,1.0
1,FIVEOFF,fixed,5.0,1,2025-08-25 17:10:01,None,10.0,NaN,NaN
2,SUMMER30,percent,30.0,1,2025-08-25 17:10:01,2025-09-25 17:10:01,30.0,2.0,NaN


Заказы (subtotal/discount/total):


,order_id,email,status,subtotal,discount_amount,total_amount,coupon
0,1,alice@example.com,paid,55.0,5.5,49.5,WELCOME10
1,2,bob@example.com,paid,45.0,5.0,40.0,FIVEOFF


Редемпшны купонов:


,order_id,email,code,redeemed_at
0,1,alice@example.com,WELCOME10,2025-08-26 17:10:03
1,2,bob@example.com,FIVEOFF,2025-08-26 17:10:03


In [6]:
import sqlite3, pandas as pd
# заказы и купоны: ожидания по суммам (из сценария)
df_ord = pd.read_sql("""
  SELECT o.order_id, u.email, o.subtotal, o.discount_amount, o.total_amount,
         (SELECT code FROM coupons c WHERE c.coupon_id=o.coupon_id) AS coupon
  FROM orders o JOIN users u ON o.user_id=u.user_id
  ORDER BY o.order_id
""", conn)
display(df_ord)
# ожидания: Alice: 2*22.5 + 10 = 55.00 → WELCOME10 = 5.50 → total 49.50
#           Bob: 1*45 = 45.00 → FIVEOFF = 5.00 → total 40.00
assert round(df_ord.loc[0,"subtotal"],2) == 55.00 and round(df_ord.loc[0,"discount_amount"],2) == 5.50 and round(df_ord.loc[0,"total_amount"],2) == 49.50
assert round(df_ord.loc[1,"subtotal"],2) == 45.00 and round(df_ord.loc[1,"discount_amount"],2) == 5.00 and round(df_ord.loc[1,"total_amount"],2) == 40.00

#  Soft-delete защита: добавление MUG в заказ должно падать
try:
    conn.execute("""
      INSERT INTO order_items(order_id, product_id, qty)
      VALUES ((SELECT MIN(order_id) FROM orders),
              (SELECT product_id FROM products WHERE sku='SKU-MUG'), 1)
    """)
    print("ОШИБКА: получилось добавить soft-deleted товар (не должно).")
except sqlite3.DatabaseError as e:
    print("Ок, защита работает:", e)

,order_id,email,subtotal,discount_amount,total_amount,coupon
0,1,alice@example.com,55.0,5.5,49.5,WELCOME10
1,2,bob@example.com,45.0,5.0,40.0,FIVEOFF


Ок, защита работает: product is soft-deleted
